# Install packages

In [ ]:
# !pip install apache-beam[gcp] google-cloud-pubsub bokeh

In [ ]:
# !pip install -e "git+https://github.com/ostrokach/beam.git@develop#egg=apache-beam&subdirectory=sdks/python"

# Imports

Packages available by default:

In [ ]:
import atexit
import json
import os
import os.path as op
import string

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
from google.cloud import bigquery, pubsub_v1

import apache_beam as beam
from apache_beam.runners.direct import direct_runner
from apache_beam.runners.interactive import interactive_runner
from apitools.base.py.exceptions import HttpConflictError

# Parameters

In [ ]:
# auth.authenticate_user()
# print('Authenticated')

In [ ]:
NOTEBOOK_NAME = "bokeh_examples"
NOTEBOOK_PATH = op.realpath(NOTEBOOK_NAME)

NOTEBOOK_PATH

# Functions

In [ ]:
class AverageFn(beam.CombineFn):
    def create_accumulator(self):
        return (0.0, 0.0, 0)

    def add_input(self, sum_count, input):
        from datetime import datetime

        (passenger_count_sum, timepoint_sum, count) = sum_count

        try:
            timestamp = datetime.strptime(
                input["timestamp"], "%Y-%m-%dT%H:%M:%S.%f-04:00"
            )
        except ValueError:
            timestamp = datetime.strptime(input["timestamp"], "%Y-%m-%dT%H:%M:%S-04:00")

        passenger_count_sum += input["passenger_count"]
        timepoint_sum += float(timestamp.strftime("%s"))
        count += 1
        return passenger_count_sum, timepoint_sum, count

    def merge_accumulators(self, accumulators):
        passenger_count_sums, timepoint_sums, counts = zip(*accumulators)
        return sum(passenger_count_sums), sum(timepoint_sums), sum(counts)

    def extract_output(self, sum_count):
        (passenger_count_sum, timepoint_sum, count) = sum_count
        passenger_count_avg = passenger_count_sum / count if count else float("NaN")
        timepoint_avg = timepoint_sum / count if count else float("NaN")
        return passenger_count_avg, timepoint_avg

In [ ]:
def update_plot(values):
    x_lst = []
    y_lst = []
    count = 0
    for element in values:
        try:
            x_lst.append(element["x"])
            y_lst.append(element["y"])
            count += 1
            if count > 10:
                break
        except (KeyError, TypeError):
            output.append(element)
    source.stream({"x": x_lst, "y": y_lst})
    push_notebook(handle=t)

In [ ]:
def increment_counter(element):
    global counter
    counter += 1
    return element

In [ ]:
def tee_to_output(element):
    output.append(element)
    return element

In [ ]:
def load_json(element):
    ju = json.loads(element)
    js = {}
    for k, v in ju.items():
        if isinstance(k, unicode):
            k = str(k)
        if isinstance(v, unicode):
            v = str(v)
        js[k] = v
    return js


assert load_json(u'{"a": 10, "b": "20"}') == {"a": 10, "b": "20"}

In [ ]:
def dump_json(element):
    element_str = json.dumps(element).encode("utf-8")
    return element_str


assert dump_json({"a": 10, "b": "20"}) == u'{"a": 10, "b": "20"}'

In [ ]:
def geographic_to_utm(longitude, latitude):
    from pyproj import Proj, transform

    x, y = transform(
        Proj(init='epsg:4326'),
        Proj(init='epsg:3857'),
        longitude,
        latitude,
    )

    return x, y

## DoFns

In [ ]:
class SelectWithinGeographicRange(beam.DoFn):
    def __init__(self, longitude_range, latitude_range):
        self.longitude_range = longitude_range
        self.latitude_range = latitude_range

    def process(self, element):
        if (
            self.longitude_range[0] <= element["longitude"] <= self.longitude_range[1]
        ) and (self.latitude_range[0] <= element["latitude"] <= self.latitude_range[1]):
            return [element]
        else:
            return []


el = {"longitude": 0, "latitude": 0}
assert SelectWithinGeographicRange((0, 1), (0, 1)).process(el) == [el]

el = {"longitude": 0, "latitude": 1}
assert SelectWithinGeographicRange((0, 1), (0, 1)).process(el) == [el]

el = {"longitude": 0, "latitude": -0.1}
assert SelectWithinGeographicRange((0, 1), (0, 1)).process(el) == []

In [ ]:
class Limit(beam.CombineFn):
    
    def __init__(self, limit=1000):
        self.limit = 1000
        
    def create_accumulator(self):
        lst = []
        return lst
    
    def add_input(self, lst, input):
        if len(lst) < self.limit:
            lst.append(input)
        return lst

    def merge_accumulators(self, accumulators):
        lst = [l for lst in accumulators for l in lst]
        lst = lst[:self.limit]
        return lst

    def extract_output(self, lst):
        return lst

In [ ]:
def add_mercator_coords(element):
    def geographic_to_utm(longitude, latitude):
        from pyproj import Proj, transform

        x, y = transform(
            Proj(init="epsg:4326"), Proj(init="epsg:3857"), longitude, latitude
        )
        return x, y

    element["x"], element["y"] = geographic_to_utm(
        element["longitude"], element["latitude"]
    )
    return element

# Parameters

In [ ]:
LONGITUDE_RANGE = (-74.07, -73.90)
LATITUDE_RANGE = (40.74, 40.76)

In [ ]:
x_min, y_min = geographic_to_utm(longitude=LONGITUDE_RANGE[0], latitude=LATITUDE_RANGE[0])
x_max, y_max = geographic_to_utm(longitude=LONGITUDE_RANGE[1], latitude=LATITUDE_RANGE[1])

MERCATOR_X_RANGE = (x_min, x_max)
MERCATOR_Y_RANGE = (y_min, y_max)

# Pipelines

### Copy `taxirides-realtime`

In [ ]:
runner = beam.runners.dataflow.DataflowRunner()

In [ ]:
pipeline_options = beam.pipeline.PipelineOptions(
    project=project_id,
    temp_location="gs://strokach/temp",
    job_name="taxirides-realtime-3",
    streaming=True,
    sdk_location=op.expanduser(
        "~/workspace/beam/sdks/python/dist/apache-beam-2.14.0.dev0.tar.gz"
    ),
)

In [ ]:
taxirides_wf = (
    beam.Pipeline(runner=runner, options=pipeline_options)
    | "Read"
    >> beam.io.ReadFromPubSub(
        topic="projects/pubsub-public-data/topics/taxirides-realtime"
    )
    | "Write"
    >> beam.io.WriteToPubSub(
        topic="projects/strokach-playground/topics/taxirides-realtime"
    )
)

In [ ]:
try:
    taxirides_r = taxirides_wf.pipeline.run()
except HttpConflictError as e:
    print(e.content)
    pass
# atexit.register(taxirides_r.cancel)

### Select subset of `taxirides-realtime`

In [ ]:
pipeline_options = beam.pipeline.PipelineOptions(
    project=project_id,
    temp_location="gs://strokach/temp",
    job_name="taxirides-realtime-ny-11",
    streaming=True,
    sdk_location=op.expanduser(
        "~/workspace/beam/sdks/python/dist/apache-beam-2.14.0.dev0.tar.gz"
    ),
    setup_file="./setup.py",
)
pipeline_options.display_data()

In [ ]:
taxirides_wf = (
    beam.Pipeline(runner=runner, options=pipeline_options)
    | "Read"
    >> beam.io.ReadFromPubSub(
        topic="projects/pubsub-public-data/topics/taxirides-realtime",
        timestamp_attribute="ts",
    )
    | "Load JSON" >> beam.Map(load_json)
    | "Window Into" >> beam.WindowInto(beam.window.FixedWindows(10))
    | "Filter coords"
    >> beam.ParDo(SelectWithinGeographicRange(LONGITUDE_RANGE, LATITUDE_RANGE))
    | "Add UTM coords" >> beam.Map(add_mercator_coords)
    #     | "Limit" >> beam.CombineGlobally(Limit(10)).without_defaults()
    | "Serialize to JSON" >> beam.Map(dump_json)
    | "Write"
    >> beam.io.WriteToPubSub(
        topic="projects/strokach-playground/topics/taxirides-realtime-ny"
    )
)

In [ ]:
try:
    taxirides_r = taxirides_wf.pipeline.run()
except HttpConflictError as e:
    print(e.content)
    pass
# atexit.register(taxirides_r.cancel)

# Dashboards

## Imports

In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
output_notebook()

## Test interactive plotting

In [ ]:
p1 = figure(plot_width=250, plot_height=250)
r1 = p1.circle([1,2,3], [4,5,6], size=20)

p2 = figure(plot_width=250, plot_height=250)
r2 = p2.circle([1,2,3], [4,5,6], size=20)

t = show(row(p1, p2), notebook_handle=True)

output_notebook()

In [ ]:
r1.glyph.fill_color = "white"
push_notebook(handle=t)

## Map taxi trips

In [ ]:
from bokeh.plotting import figure, show
from bokeh.tile_providers import Vendors, get_provider

# range bounds supplied in web mercator coordinates
p = figure(
    x_range=MERCATOR_X_RANGE,
    y_range=MERCATOR_Y_RANGE,
    x_axis_type="mercator",
    y_axis_type="mercator",
#     plot_height=600,
)
p.add_tile(get_provider(Vendors.CARTODBPOSITRON))

source = ColumnDataSource(data=dict(x=[], y=[]))

p.circle(x="x", y="y", size=6, fill_color="blue", fill_alpha=0.8, source=source)

t = show(p, notebook_handle=True)

In [ ]:
# source.stream({"x": [0,1,2], "y": [1,2,3]})
# push_notebook(handle=t)

In [ ]:
from google.cloud import pubsub_v1

In [ ]:
def update_plot(message):
    global count
    output.append(json.loads(message.data))
    count += 1
#     x = json.loads(message.data)['x']
#     y = json.loads(message.data)['y']
#     source.stream({"x": [x], "y": [y]})
#     push_notebook(handle=t)

In [ ]:
raise Exception

In [ ]:
import time

from google.cloud import pubsub_v1

subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path(project_id, "noo")

count = 0
output = []


def callback(message):
    print("Received message: {}".format(message.data))
    outputs.append(message)
#     message.ack()

flow_control = pubsub_v1.types.FlowControl(max_messages=10)
future = subscriber.subscribe(subscription_path, callback=update_plot, flow_control=flow_control)
result = future.result(timeout=10)
# The subscriber is non-blocking. We must keep the main thread from
# exiting to allow it to process messages asynchronously in the background.
print("Listening for messages on {}".format(subscription_path))
# while True:
#     try:
#         future.result(timeout=2)
#     except Exception as e:
#         print(e)
#     subscriber.subscribe(subscription_path, callback=update_plot)

#     print(len(outputs))

In [ ]:

while True:
    subscriber = pubsub_v1.SubscriberClient()
    try:
        subscription_path = subscriber.subscription_path(project_id, "noo")
        pull_response = subscriber.pull(subscription_path, max_messages=100, timeout=5, retry=None)
        output = []
        for msg in pull_response.received_messages:
            output.append(msg)
        print(len(output))
    finally:
        subscriber.api.transport._channel.close()


In [ ]:
count

In [ ]:
pubsub_v1.types.FlowControl?

In [ ]:
project = subscriber.project_path(project_id)
list(subscriber.list_subscriptions(project))

In [ ]:
json.loads(outputs[0].data)['x']
json.loads(outputs[0].data)['x']

# Things that did not work

### Read stream using `DirectRunner`

In [ ]:
output_folder = op.join(NOTEBOOK_PATH, "pipeline-output") 

try:
    os.makedirs(output_folder)
except OSError:
    pass

In [ ]:
output = []
counter = 0

data = (
    beam.Pipeline(
        runner="direct", options=beam.pipeline.PipelineOptions(streaming=True)
    )
    | "Read"
    >> beam.io.ReadFromPubSub(
        topic="projects/strokach-playground/topics/taxirides-realtime-ny",
        #         subscription="projects/strokach-playground/subscriptions/beam_1558483403_228577dd",
        timestamp_attribute="ts",
    )
    | "Load JSON" >> beam.Map(load_json)
#     | "Window Into" >> beam.WindowInto(beam.window.FixedWindows(10))
#     | "Filter non-NY" >> beam.FlatMap(filter_ny)
#     | "Add UTM coords" >> beam.Map(geographic_to_utm)
#     | "Update plot" >> beam.CombineGlobally(update_plot).without_defaults()
    | "Tee to output" >> beam.Map(tee_to_output)
    | "Update counter" >> beam.Map(increment_counter)
)

In [ ]:
result = data.pipeline.run()
# atexit.register(result.cancel)

In [ ]:
result.cancel()

In [ ]:
result.state

In [ ]:
counter

In [ ]:
output

In [ ]:
# result.cancel()

#### Errors

After ~30 seconds, the process crashes with errors:

```python
The history saving thread hit an unexpected error (OperationalError('unable to open database file',)).
History will not be written to the database.
```

```python
ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7f61ea61cc50>" raised exception!
Traceback (most recent call last):
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/grpc/_plugin_wrapping.py", line 80, in __call__
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/google/auth/transport/grpc.py", line 77, in __call__
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/google/auth/transport/grpc.py", line 65, in _get_authorization_headers
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/google/auth/credentials.py", line 122, in before_request
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/google/oauth2/service_account.py", line 322, in refresh
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/google/oauth2/_client.py", line 145, in jwt_grant
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/google/oauth2/_client.py", line 106, in _token_endpoint_request
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/google/auth/transport/requests.py", line 124, in __call__
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/six.py", line 737, in raise_from
TransportError: HTTPSConnectionPool(host='oauth2.googleapis.com', port=443): Max retries exceeded with url: /token
            (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f61ea62d810>: Failed to establish a new connection: [Errno 24] Too many open files',))
```
            
```python
Traceback (most recent call last):
Exception in thread Thread-490:
Traceback (most recent call last):
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/threading.py", line 801, in __bootstrap_inner
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/threading.py", line 754, in run
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/grpc/_plugin_wrapping.py", line 84, in __call__
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/logging/__init__.py", line 1207, in exception
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/logging/__init__.py", line 1200, in error
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/logging/__init__.py", line 1293, in _log
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/logging/__init__.py", line 1303, in handle
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/logging/__init__.py", line 1343, in callHandlers
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/logging/__init__.py", line 766, in handle
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/logging/__init__.py", line 896, in emit
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/logging/__init__.py", line 819, in handleError
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/traceback.py", line 124, in print_exception
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/traceback.py", line 13, in _print
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/ipykernel/iostream.py", line 400, in write
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/ipykernel/iostream.py", line 203, in schedule
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/ipykernel/iostream.py", line 101, in _event_pipe
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/zmq/sugar/context.py", line 146, in socket
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/zmq/sugar/socket.py", line 59, in __init__
  File "zmq/backend/cython/socket.pyx", line 328, in zmq.backend.cython.socket.Socket.__init__
ZMQError: Too many open files
Unhandled exception in thread started by <bound method Thread.__bootstrap of <Thread(Thread-490, stopped daemon 140058531190528)>>
```

```python
Unhandled exception in thread started by 
Traceback (most recent call last):
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/threading.py", line 774, in __bootstrap
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/threading.py", line 814, in __bootstrap_inner
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/ipykernel/iostream.py", line 400, in write
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/ipykernel/iostream.py", line 203, in schedule
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/ipykernel/iostream.py", line 101, in _event_pipe
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/zmq/sugar/context.py", line 146, in socket
  File "/home/strokach/miniconda3/envs/beam-dev/lib/python2.7/site-packages/zmq/sugar/socket.py", line 59, in __init__
  File "zmq/backend/cython/socket.pyx", line 328, in zmq.backend.cython.socket.Socket.__init__
ZMQError: Too many open files
```

```
ZMQbg/0    3922  3934   strokach  753u  a_inode               0,11        0       8555 [eventpoll]
ZMQbg/0    3922  3934   strokach  754u  a_inode               0,11        0       8555 [eventfd]
ZMQbg/0    3922  3934   strokach  755u     IPv6             916106      0t0        TCP workstation.c.strokach-playground.internal:40042->lax28s15-in-f10.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  756u     IPv4             915269      0t0        TCP workstation.c.strokach-playground.internal:40044->lax28s15-in-f10.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  757u  a_inode               0,11        0       8555 [eventpoll]
ZMQbg/0    3922  3934   strokach  758u  a_inode               0,11        0       8555 [eventfd]
ZMQbg/0    3922  3934   strokach  759u     IPv6             916116      0t0        TCP workstation.c.strokach-playground.internal:43134->lax17s15-in-f74.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  760u     IPv4             916121      0t0        TCP workstation.c.strokach-playground.internal:48062->lax28s10-in-f10.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  761u  a_inode               0,11        0       8555 [eventpoll]
ZMQbg/0    3922  3934   strokach  762u  a_inode               0,11        0       8555 [eventfd]
ZMQbg/0    3922  3934   strokach  763u     IPv6             916125      0t0        TCP workstation.c.strokach-playground.internal:34964->lax17s14-in-f138.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  764u     IPv4             916882      0t0        TCP workstation.c.strokach-playground.internal:34966->lax17s14-in-f138.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  765u  a_inode               0,11        0       8555 [eventpoll]
ZMQbg/0    3922  3934   strokach  766u  a_inode               0,11        0       8555 [eventfd]
ZMQbg/0    3922  3934   strokach  767u     IPv6             915304      0t0        TCP workstation.c.strokach-playground.internal:43082->lax17s05-in-f10.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  768u     IPv4             910231      0t0        TCP workstation.c.strokach-playground.internal:45014->lax17s38-in-f10.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  769u  a_inode               0,11        0       8555 [eventpoll]
ZMQbg/0    3922  3934   strokach  770u  a_inode               0,11        0       8555 [eventfd]
ZMQbg/0    3922  3934   strokach  771u     IPv6             916128      0t0        TCP workstation.c.strokach-playground.internal:43086->lax17s05-in-f10.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  772u     IPv4             916132      0t0        TCP workstation.c.strokach-playground.internal:40060->lax28s15-in-f10.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  773u  a_inode               0,11        0       8555 [eventpoll]
ZMQbg/0    3922  3934   strokach  774u  a_inode               0,11        0       8555 [eventfd]
ZMQbg/0    3922  3934   strokach  775u     IPv6             916136      0t0        TCP workstation.c.strokach-playground.internal:40062->lax28s15-in-f10.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  776u     IPv4             915310      0t0        TCP workstation.c.strokach-playground.internal:46920->lax02s23-in-f10.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  777u  a_inode               0,11        0       8555 [eventpoll]
ZMQbg/0    3922  3934   strokach  778u  a_inode               0,11        0       8555 [eventfd]
ZMQbg/0    3922  3934   strokach  779u     IPv6             910235      0t0        TCP workstation.c.strokach-playground.internal:43154->lax17s15-in-f74.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  780u     IPv4             916139      0t0        TCP workstation.c.strokach-playground.internal:46924->lax02s23-in-f10.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  781u  a_inode               0,11        0       8555 [eventpoll]
ZMQbg/0    3922  3934   strokach  782u  a_inode               0,11        0       8555 [eventfd]
ZMQbg/0    3922  3934   strokach  783u     IPv6             916888      0t0        TCP workstation.c.strokach-playground.internal:46926->lax02s23-in-f10.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  784u     IPv4             916149      0t0        TCP workstation.c.strokach-playground.internal:34986->lax17s14-in-f138.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  785u  a_inode               0,11        0       8555 [eventpoll]
ZMQbg/0    3922  3934   strokach  786u  a_inode               0,11        0       8555 [eventfd]
ZMQbg/0    3922  3934   strokach  787u     IPv6             915368      0t0        TCP workstation.c.strokach-playground.internal:34988->lax17s14-in-f138.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  788u     IPv4             916153      0t0        TCP workstation.c.strokach-playground.internal:45034->lax17s38-in-f10.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  789u  a_inode               0,11        0       8555 [eventpoll]
ZMQbg/0    3922  3934   strokach  790u  a_inode               0,11        0       8555 [eventfd]
ZMQbg/0    3922  3934   strokach  791u     IPv6             916157      0t0        TCP workstation.c.strokach-playground.internal:43106->lax17s05-in-f10.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  792u     IPv4             916891      0t0        TCP workstation.c.strokach-playground.internal:40080->lax28s15-in-f10.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  793u  a_inode               0,11        0       8555 [eventpoll]
ZMQbg/0    3922  3934   strokach  794u  a_inode               0,11        0       8555 [eventfd]
ZMQbg/0    3922  3934   strokach  795u     IPv6             916161      0t0        TCP workstation.c.strokach-playground.internal:43170->lax17s15-in-f74.1e100.net:https (ESTABLISHED)
ZMQbg/0    3922  3934   strokach  796u     IPv4             915373      0t0        TCP workstation.c.strokach-playground.internal:46940->lax02s23-in-f10.1e100.net:https (ESTABLISHED)
```

In [ ]:
output[0]['y']

In [ ]:
counter

In [ ]:
output

In [ ]:
from bokeh.plotting import figure, show
from bokeh.tile_providers import get_provider, Vendors

# range bounds supplied in web mercator coordinates
p = figure(x_range=(-1000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(get_provider(Vendors.CARTODBPOSITRON))

source = ColumnDataSource(
    data=dict(lat=[ 30.29,  30.20,  30.29],
              lon=[-97.70, -97.74, -97.78])
)

p.circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, source=source)

show(p)

In [ ]:
def modify_doc(doc):
    source = ColumnDataSource(data=dict(x=[], y=[]))

    p = figure(
        x_range=MERCATOR_X_RANGE,
        y_range=MERCATOR_Y_RANGE,
        x_axis_type="mercator",
        y_axis_type="mercator",
        plot_height=800,
    )
    p.add_tile(get_provider(Vendors.CARTODBPOSITRON))
    p.circle(x="x", y="y", size=2, fill_color="blue", fill_alpha=0.8, source=source)

    while True:
        for i, element in enumerate(temp.read()):
            source.stream({"x": [element["x"]], "y": [element["y"]]})
        
    doc.add_root(p)

    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 800
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """))

In [ ]:
show(modify_doc) # notebook_url="http://localhost:8888"

In [ ]:
import yaml

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.io import show, output_notebook

from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature

output_notebook()


In [ ]:
def modify_doc(doc):
    df = sea_surface_temperature.copy()
    source = ColumnDataSource(data=df)

    plot = figure(x_axis_type='datetime', y_range=(0, 25),
                  y_axis_label='Temperature (Celsius)',
                  title="Sea Surface Temperature at 43.18, -70.43")
    plot.line('time', 'temperature', source=source)

    def callback(attr, old, new):
        if new == 0:
            data = df
        else:
            data = df.rolling('{0}D'.format(new)).mean()
        source.data = ColumnDataSource(data=data).data

    slider = Slider(start=0, end=30, value=0, step=1, title="Smoothing by N Days")
    slider.on_change('value', callback)

    doc.add_root(column(slider, plot))

    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """))


In [ ]:
show(modify_doc) # notebook_url="http://localhost:8888"

In [ ]:
counter